In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import random

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/115.0.0.0 Safari/537.36"
}

In [ ]:
# URL
base_url = "https://myanimelist.net/topanime.php"

In [16]:
anime_data = {
    "ranking": [],
    "nombre": [],
    "puntuacion": [],
    "votos": [],
    "tipo": [],
    "episodios": [],
    "duracion": [],
    "demografia": [],
    "source": [],
    "rating_edad": [],
    "miembros": [],
    "favoritos": []
}

In [ ]:
# Scrapear 500 animes (50 por página)
for i in range(0, 500, 50):
    print(f"Scrapeando página {i // 50 + 1} (offset {i})...")
    url = f"{base_url}?limit={i}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.find_all("tr", class_="ranking-list")

    for row in rows:
        try:
            ranking = row.find("td", class_="rank").text.strip()
            title_tag = row.find("h3", class_="anime_ranking_h3").find("a")
            title = title_tag.text.strip()
            anime_url = title_tag["href"]

            score_text = row.find("td", class_="score").text.strip()
            score = score_text if "." in score_text else score_text[:-2] + "." + score_text[-2:]

            # Retry al acceder a la página del anime
            for intento in range(3):
                anime_resp = requests.get(anime_url, headers=headers)
                if anime_resp.status_code == 200:
                    break
                print(f"Reintentando acceso a {title} (intento {intento + 1})")
                time.sleep(2)
            else:
                print(f"Falló acceso a {title} tras 3 intentos. URL: {anime_url}")
                continue

            anime_soup = BeautifulSoup(anime_resp.text, "html.parser")

            tipo_tag = anime_soup.find("span", string="Type:")
            tipo = tipo_tag.find_next("a").text if tipo_tag else ""

            episodios_tag = anime_soup.find("span", string="Episodes:")
            episodios = episodios_tag.next_sibling.strip() if episodios_tag else ""

            duracion_tag = anime_soup.find("span", string="Duration:")
            duracion = duracion_tag.next_sibling.strip() if duracion_tag else ""

            rating_tag = anime_soup.find("span", string="Rating:")
            rating_edad = rating_tag.next_sibling.strip() if rating_tag else ""

            miembros_tag = anime_soup.find("span", string="Members:")
            miembros = miembros_tag.next_sibling.strip() if miembros_tag else ""

            favoritos_tag = anime_soup.find("span", string="Favorites:")
            favoritos = favoritos_tag.next_sibling.strip() if favoritos_tag else ""

            source_tag = anime_soup.find("span", string="Source:")
            source = ""
            if source_tag:
                link = source_tag.find_next("a")
                if link:
                    source = link.text.strip()
                else:
                    sibling = source_tag.next_sibling
                    if sibling:
                        source = sibling.strip() if isinstance(sibling, str) else sibling.get_text(strip=True)
            
            score_stat_tag = anime_soup.find("span", string=re.compile("scored by"))
            votos = ""
            if score_stat_tag:
                match = re.search(r"scored by ([\d,]+) users", score_stat_tag.text)
                votos = match.group(1) if match else ""

            demografia_tag = anime_soup.find("span", string="Demographic:")
            demografia = ""
            if demografia_tag:
                demografia_a = demografia_tag.find_next("a")
                demografia = demografia_a.text.strip() if demografia_a else ""

            if not title or not tipo:
                print(f"Datos incompletos para ranking {ranking}, se omite.")
                continue

            # Guardar 
            anime_data["ranking"].append(ranking)
            anime_data["nombre"].append(title)
            anime_data["puntuacion"].append(score)
            anime_data["votos"].append(votos)
            anime_data["tipo"].append(tipo)
            anime_data["episodios"].append(episodios)
            anime_data["duracion"].append(duracion)
            anime_data["demografia"].append(demografia)
            anime_data["source"].append(source)
            anime_data["rating_edad"].append(rating_edad)
            anime_data["miembros"].append(miembros)
            anime_data["favoritos"].append(favoritos)

            # Esperar para no ser bloqueado
            time.sleep(random.uniform(1.0, 1.5))

        except Exception as e:
            print(f"Error en anime #{ranking} ({title if 'title' in locals() else 'desconocido'}): {e}")
            continue


Scrapeando página 1 (offset 0)...
Scrapeando página 2 (offset 50)...
Scrapeando página 3 (offset 100)...
Scrapeando página 4 (offset 150)...
Scrapeando página 5 (offset 200)...
Scrapeando página 6 (offset 250)...
Scrapeando página 7 (offset 300)...
Scrapeando página 8 (offset 350)...
Scrapeando página 9 (offset 400)...
Scrapeando página 10 (offset 450)...


In [ ]:
df_animes = pd.DataFrame(anime_data)
df_animes["ranking"] = range(1, len(df_animes) + 1)  
df_animes.to_csv("myanimelist_top500.csv", index=False, encoding="utf-8")
